In [1]:
import sys
sys.path.append('../src')

from wikidataDB import Session, WikidataID, WikidataEntity
from wikidataEmbed import WikidataTextifier, JinaAIEmbeddings
from tqdm import tqdm
import json
import os
import torch
from langchain_core.documents import Document
from ragstack_langchain.graph_store import CassandraGraphStore
import cassio

In [2]:
datastax_token = json.load(open("../API tokens/datastax_token.json"))
os.environ["ASTRA_DB_DATABASE_ID"] = datastax_token['database_id']
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = datastax_token['token']

BATCH_SIZE = 1
BATCH_LENGTH = 5000

cassio.init(auto=True)
embeddings = JinaAIEmbeddings(embedding_dim=1024)
textifier = WikidataTextifier(with_claim_aliases=True, with_property_aliases=False)
graph_store = CassandraGraphStore(
    embeddings,
    node_table="Wikidata_entities_v1",
)

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [3]:
with open("../data/Wikidata/pushed_embeddings.json", "r+") as file:
    json_str = file.read().strip()
    if json_str.endswith(','):
        json_str = json_str[:-1]+"}"
    prev_data = json.loads(json_str)

In [4]:
offset = len([k for k,q in prev_data.items() if "_1" in k]) -5
offset

93578

In [6]:
with tqdm(total=9203531) as progressbar:
    with Session() as session:
        entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(offset).yield_per(1000)
        doc_batch = []
        batch_length = 0
        progressbar.update(offset)

        for entity in entities:
            progressbar.update(1)
            chunks = embeddings.chunk_text(entity, textifier)
            for chunk_i in range(len(chunks)):
                doc = Document(page_content=chunks[chunk_i], metadata={"QID": entity.id, "ChunkID": chunk_i+1})
                if f"{entity.id}_{chunk_i+1}" not in prev_data:
                    doc_batch.append(doc)
                    batch_length += len(chunks[chunk_i])

                if batch_length >= BATCH_LENGTH:
                    try:
                        graph_store.add_documents(doc_batch)
                        torch.cuda.empty_cache()
                        with open("../data/Wikidata/pushed_embeddings.json", "a+") as file:
                            file.write(", ".join([f"\"{d.metadata["QID"]}_{d.metadata["ChunkID"]}\": 1" for d in doc_batch]) +", ")

                        progressbar.set_description(f"Batch Size: {len(doc_batch)}")
                        doc_batch = []
                        batch_length = 0
                    except Exception as e:
                        torch.cuda.empty_cache()
                        progressbar.set_description(f"Batch Size: 1")
                        print(e)

                        while len(doc_batch) > 0:
                            doc = doc_batch.pop()
                            embeddings.model = embeddings.model.to('cpu')
                            graph_store.add_documents([doc])
                            torch.cuda.empty_cache()
                            with open("../data/Wikidata/pushed_embeddings.json", "a+") as file:
                                file.write(f"\"{doc.metadata["QID"]}_{doc.metadata["ChunkID"]}\": 1, ")
                        batch_length = 0
                        embeddings.model = embeddings.model.to('cuda')

  1%|          | 93583/9203531 [1:33:29<151:41:30, 16.68it/s]


KeyboardInterrupt: 

In [5]:
import torch
torch.cuda.empty_cache()

In [32]:
vector_retriever = graph_store.as_retriever(search_kwargs={"k": 1000, "depth": 0})
results = vector_retriever.get_relevant_documents("Is this a question?")

In [6]:
cassio.init(auto=True)
from cassio.config import check_resolve_keyspace, check_resolve_session

session = check_resolve_session()
keyspace = check_resolve_keyspace()

In [7]:
session.execute(f"DROP TABLE IF EXISTS {keyspace}.wikidata_entities")